In [ ]:
!sudo apt -y install python3-magic
!pip install tfds-nightly
!pip install augly

In [ ]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import augly.image as imaugs

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import scipy
from sklearn.metrics import roc_auc_score
import pickle
import gc
import PIL
from tensorflow.keras.applications import *
import imagehash
from PIL import Image
from tensorflow.keras import mixed_precision

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
BATCH_SIZE = 32

In [ ]:
class ReAct(tf.keras.layers.Layer):
  def __init__(self, λ):
    super(ReAct, self).__init__()
    self.λ = λ
  def build(self, input_shape):
    pass
  def call(self, inputs):
    return tf.math.minimum(inputs, self.λ)

In [ ]:
def get_model(ds, model_base, preprocessing, p=0.9, show_info=False):
    pos = -1
    while True:
        layer_fv = model_base.get_layer(index=pos)
        if layer_fv.output.shape[-1] != model_base.get_layer(index=-1).output.shape[-1]:
            break
        pos -= 1

    model_fv = tf.keras.Model(inputs=model_base.input, outputs=layer_fv.output)
    model_fv = tf.keras.Sequential([model_fv, tf.keras.layers.Flatten()])
    y = model_fv.predict(ds).flatten()
    y_pos = min(int(p*len(y)), len(y)-1)
    λ = np.partition(y, pos, axis=0)[y_pos]
    if show_info:
        print(f'λ: {λ}')
    
    inp = tf.keras.layers.Input(shape=model_fv.output.shape[1:])
    x = inp
    for p in range(pos+1, 0):
        layer = model_base.get_layer(index=p)
        try:
            if layer.activation == tf.keras.activations.softmax:
                layer.activation = tf.keras.activations.linear
                if show_info:
                    print('softmax layer removed')
        except:
            pass
        x = layer(x)
    
    last_block = tf.keras.Model(inputs=inp, outputs=x)
    last_block.compile('adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    @tf.function(jit_compile=True)
    def prep_image(x):
        return preprocessing(255.0*x)

    layer_prep_image = tf.keras.layers.Lambda(prep_image)
    # feature vector
    model_react = tf.keras.Sequential([
        tf.keras.Input((224, 224, 3)),
        layer_prep_image,
        model_fv,
        ReAct(λ),
        last_block
    ])
    
    model_base = tf.keras.Sequential([
        tf.keras.Input((224, 224, 3)),
        layer_prep_image,
        model_base,
    ])

    model_react.compile('adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'], steps_per_execution=64)
    model_base.compile('adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'], steps_per_execution=64)
    fe_base = tf.keras.Sequential([
        model_fv,        
    ])
    
    fe_react = tf.keras.Sequential([
        model_fv,  
        ReAct(λ),
    ])
    
    return model_base, model_react, fe_base, fe_react

In [ ]:
def get_batch_gradient(model, loss_object, x, y_true=None):
  x = tf.convert_to_tensor(x)
  with tf.GradientTape() as tape:
    tape.watch(x)
    y_pred = model(x)
    if y_true is None:
      loss = loss_object(y_pred)
    else:
      loss = loss_object(y_true, y_pred)
  gradient = tape.gradient(loss, x)
  del loss, loss_object, x
  return gradient.numpy()

def get_gradient(model, loss_object, x, y_true=None, batch_size=BATCH_SIZE):
  grad = []
  pos = 0
  while pos < len(x):
    if y_true is not None:
      y_true_batch = y_true[pos:pos+batch_size]
    else:
      y_true_batch = None
    grad.append(get_batch_gradient(model, loss_object, x[pos:pos+batch_size], y_true=y_true_batch))
    pos += batch_size
  return np.concatenate(grad, axis=0)

def get_softmax_score(model, x, t=1):
  y_pred = model.predict(x)/t
  y_pred = scipy.special.softmax(y_pred, axis=0)
  y_pred = np.max(y_pred, axis=1)
  return y_pred

def odin_perturbation(model, x, t, epsilon):
  def loss(y_pred):
    y_pred = y_pred / t
    y_pred =  tf.nn.softmax(y_pred, axis=-1)
    return tf.math.log(tf.math.reduce_max(y_pred, axis=0))
  g = get_gradient(model, loss, x.copy())
  x -= epsilon * np.sign(-g)
  return x

def get_odin_score(model, x, t=1, epsilon=0.004):
  x = odin_perturbation(model, x, t=t, epsilon=epsilon)
  y_pred = model.predict(x)/t
  y_pred = scipy.special.softmax(y_pred)
  y_pred = np.max(y_pred, axis=1)
  return y_pred

def get_energy_score(model, x):
  y_pred = model.predict(x)
  score = np.log(1e-6+np.sum(np.exp(y_pred), axis=1))
  return score

In [ ]:
def test_accuracy(model):
    ds = tfds.load('imagenet_v2', split=f'test[:{N_IMAGES}]', as_supervised=True, try_gcs=True)
    ds = ds.map(lambda x, y: (tf.image.resize(tf.image.convert_image_dtype(x, tf.float32), (224, 224)), y)) # [0, 1]
    ds = ds.batch(BATCH_SIZE)
    print(model.evaluate(ds))

In [ ]:
def create_adversarial_pattern(model, input_image, loss_object):
    gradient = get_gradient(model, loss_object, input_image, y_true=None, batch_size=BATCH_SIZE)
    signed_grad = tf.sign(gradient)
    return signed_grad

# Run

In [ ]:
N_IMAGES = 512

In [ ]:
dataset = pickle.load(open('../input/preprocess-dataset/dataset.pkl', 'rb'))
dataset.keys()

In [ ]:
def get_dataset(name, split_type):
    if name == 'gauss':
        ds =  tf.data.Dataset.from_tensor_slices(np.clip(np.random.normal(size=(N_IMAGES, 224, 224, 3)).astype(np.float32), 0, 1))
    elif name == 'uniform':
        ds =  tf.data.Dataset.from_tensor_slices(np.clip(np.random.uniform(0, 1, size=(N_IMAGES, 224, 224, 3)).astype(np.float32), 0, 1))
    elif name in dataset:
        ds = tf.data.Dataset.from_tensor_slices(dataset[name][:N_IMAGES].astype(np.float32)/255)
    else:
        ds = tfds.load(name, split=f'{split_type}[:{N_IMAGES}]', as_supervised=True, try_gcs=True)
        ds = ds.map(lambda x, y: (tf.image.resize(tf.image.convert_image_dtype(x, tf.float32), (224, 224)))) # [0, 1]
    ds = ds.batch(BATCH_SIZE)
    
    return ds

def augly_wrapper(x, f, **karg):
    return np.array([imaugs.aug_np_wrapper((255*a).astype(np.uint8), f, **karg) for a in x])

In [ ]:
# '''
data = [
    ('imagenet', 'adversarial_0.1_softmax_base', 'validation'),
    ('imagenet', 'adversarial_0.1_msoftmax_base', 'validation'),
    ('imagenet', 'adversarial_0.1_softmax100_base', 'validation'),

    ('imagenet', 'adversarial_0.1_softmax_react', 'validation'),
    ('imagenet', 'adversarial_0.1_msoftmax_react', 'validation'),
    ('imagenet', 'adversarial_0.1_softmax100_react', 'validation'),
    
    ('imagenet', 'adversarial_0.01_softmax_base', 'validation'),
    ('imagenet', 'adversarial_0.01_msoftmax_base', 'validation'),
    ('imagenet', 'adversarial_0.01_softmax100_base', 'validation'),

    ('imagenet', 'adversarial_0.01_softmax_react', 'validation'),
    ('imagenet', 'adversarial_0.01_msoftmax_react', 'validation'),
    ('imagenet', 'adversarial_0.01_softmax100_react', 'validation'),

    ('imagenet_a', '', 'test'),
    ('rock_paper_scissors', '', 'train'),
    ('imagenet_v2', '', 'test'),
    ('imagenette', '', 'validation'),
    
    ('iNaturalist', '', None),
    ('SUN', '', None),
    ('Places', '', None),

    ('iNaturalist', 'adversarial_0.1_softmax_base', 'validation'),
    ('iNaturalist', 'adversarial_0.1_msoftmax_base', 'validation'),
    ('iNaturalist', 'adversarial_0.1_softmax_react', 'validation'),
    ('iNaturalist', 'adversarial_0.1_msoftmax_react', 'validation'),
    

    ('gauss', '', None),
    ('uniform', '', None),   
    
    ('imagenet', '', 'validation'),
    
    ('imagenet', 'gaussian_0.001', 'validation'),
    ('imagenet', 'gaussian_0.05', 'validation'),
    ('imagenet', 'gaussian_0.1', 'validation'),
    ('imagenet', 'gaussian_0.3', 'validation'),
    ('imagenet', 'gaussian_0.5', 'validation'),
    ('imagenet', 'gaussian_0.8', 'validation'),
    ('imagenet', 'gaussian_1.0', 'validation'),
    ('imagenet', 'gaussian_2.0', 'validation'),
    ('imagenet', 'gaussian_5.0', 'validation'),
    ('imagenet', 'gaussian_10.0', 'validation'),
    ('imagenet', 'gaussian_20.0', 'validation'),
    
    ('imagenet', 'blur_0.5', 'validation'),
    ('imagenet', 'blur_1.0', 'validation'),
    ('imagenet', 'blur_5.0', 'validation'),
    ('imagenet', 'blur_10.0', 'validation'),
    ('imagenet', 'blur_20.0', 'validation'),
    
    ('imagenet', 'pixelization_0.9', 'validation'),
    ('imagenet', 'pixelization_0.7', 'validation'),
    ('imagenet', 'pixelization_0.5', 'validation'),
    ('imagenet', 'pixelization_0.2', 'validation'),
    ('imagenet', 'pixelization_0.1', 'validation'),
    
    ('imagenet', 'perspectivetransform_10.0', 'validation'),
    ('imagenet', 'perspectivetransform_25.0', 'validation'),
    ('imagenet', 'perspectivetransform_50.0', 'validation'),
    ('imagenet', 'perspectivetransform_75.0', 'validation'),
    ('imagenet', 'perspectivetransform_100.0', 'validation'),
    
    ('imagenet', 'encodingquality_75', 'validation'),
    ('imagenet', 'encodingquality_50', 'validation'),
    ('imagenet', 'encodingquality_25', 'validation'),
    ('imagenet', 'encodingquality_15', 'validation'),
    ('imagenet', 'encodingquality_5', 'validation'),
    

    # ('malaria', '', 'train'),
]   
'''
data = [
    ('imagenet', '', 'validation'),
    ('iNaturalist', '', None),
    ('SUN', '', None),
    ('Places', '', None),
    ('imagenet_v2', '', 'test'),
    
    ('imagenet', 'gaussian_0.002', 'validation'),
    ('imagenet', 'gaussian_0.01', 'validation'),
    ('imagenet', 'gaussian_0.05', 'validation'),
    ('imagenet', 'gaussian_0.25', 'validation'),
    ('imagenet', 'gaussian_1.25', 'validation'),

    ('iNaturalist', 'adversarial_0.01_softmax_base', 'validation'),
    ('iNaturalist', 'adversarial_0.001_softmax_base', 'validation'),
    ('iNaturalist', 'adversarial_0.0001_softmax_base', 'validation'),
]
# '''

In [ ]:
output = {}
weights = {}
models_config = [
    # (ResNet50V2(input_shape=(224, 224, 3), classifier_activation='linear'), 'ResNet50V2', resnet_v2.preprocess_input),
    # (DenseNet121(input_shape=(224, 224, 3)), 'DenseNet121', densenet.preprocess_input),
    # (DenseNet169(input_shape=(224, 224, 3)), 'DenseNet169', densenet.preprocess_input),
    # (DenseNet201(input_shape=(224, 224, 3)), 'DenseNet201', densenet.preprocess_input),
    # (EfficientNetB0(input_shape=(224, 224, 3), classifier_activation='linear'), 'EfficientNetB0', efficientnet.preprocess_input),
    # (VGG16(input_shape=(224, 224, 3), classifier_activation='linear'), 'VGG16', vgg16.preprocess_input),
    # (VGG19(input_shape=(224, 224, 3), classifier_activation='linear'), 'VGG19', vgg19.preprocess_input),

    (ResNet101V2(input_shape=(224, 224, 3), classifier_activation='linear'), 'ResNet101V2', resnet_v2.preprocess_input),
    (ResNet152V2(input_shape=(224, 224, 3), classifier_activation='linear'), 'ResNet152V2', resnet_v2.preprocess_input),

]

In [ ]:
ds = get_dataset('imagenet', None)
x = np.concatenate([a.numpy() for a in ds], axis=0)
x.shape

In [ ]:
for model_base, model_name, preprocessing in models_config:
    print('\n\n\n\n'+'*'*120)
    print(model_name)

    model_base, model_react, fe_base, fe_react = get_model(ds, model_base, preprocessing, p=0.9)

    test_accuracy(model_base)
    test_accuracy(model_react)

In [ ]:
for model_base, model_name, preprocessing in models_config:
    gc.collect()
    tf.keras.backend.clear_session()
    gc.collect()

    model_base, model_react, fe_base, fe_react = get_model(ds, model_base, preprocessing, p=0.9)
    output[model_name] = {}
    weights[model_name] = model_react.get_layer(index=-1).weights

    for name, transformation, split_type in data:
        print(name, transformation, split_type)
        ds = get_dataset(name, split_type)
        x = np.concatenate([a.numpy() for a in ds], axis=0)
        x_preaug = x.copy()
        for t in transformation.split(';'):
            s = t.split('_')
            print(s)
            if s[0] == 'gaussian':
                x = augly_wrapper(x, imaugs.random_noise, **{'var': float(s[1])})
            elif s[0] == 'blur':
                x = augly_wrapper(x, imaugs.blur, **{'radius': float(s[1])})
            elif s[0] == 'pixelization':
                x = augly_wrapper(x, imaugs.pixelization, **{'ratio': float(s[1])})
            elif s[0] == 'perspectivetransform':
                x = augly_wrapper(x, imaugs.perspective_transform, **{'sigma': float(s[1])})
            elif s[0] == 'encodingquality':
                x = augly_wrapper(x, imaugs.encoding_quality, **{'quality': int(s[1])})
            elif s[0] == 'adversarial': # example: adversarial_0.03_softmax_react
                epsilon = float(s[1])
                loss = {
                    'softmax': lambda x:  tf.math.reduce_logsumexp(tf.math.softmax(x)),
                    'msoftmax': lambda x: - tf.math.reduce_logsumexp(tf.math.softmax(x)),
                    'softmax100': lambda x:  tf.math.reduce_logsumexp(tf.math.softmax(x/100)),
                }[s[2]]
                model = {
                    'react': model_react,
                    'base': model_base
                }[s[3]]

                x = (x-x.min())/(x.max()-x.min())
                dx = epsilon*create_adversarial_pattern(model, x, loss).numpy()
                x = x + dx
                x = np.clip(x, 0, 1)
            elif t == '':
                pass
            else:
                print(f'invalid transformation: {t}')

        plt.imshow(x[0])
        plt.show()

        print(x.min(), x.max(), x.dtype) # (a, b) -> (-1, 1)
        dx = x.max() - x.min()
        if abs(x.min() - (0)) <= 0.2*dx and abs(x.max() - (1)) <= 0.3*dx:
            x = 2*x.astype(np.float32)-1
        elif (abs(x.min() - (0)) <= 0.3*dx and abs(x.max() - (255)) <= 0.3*dx) or x.dtype == np.uint8:
            x = x.astype(np.float32) / 255
            x = 2*x-1
        elif abs(x.min() - (-1)) <= 0.3*dx and abs(x.max() - (1)) <= 0.3*dx:
            x = x
        else:
            print('Invalid conversion')

        # (-1, 1) -> (0, 1)
        x = (1+x)*0.5

        print(x.min(), x.max(), x.dtype)

        name = name + "|" + transformation

        output[model_name][name] = {}
        print(f'{model_name} - {name}')

        output[model_name][name]['images'] = x[:5].astype('float32')
        gc.collect()
        output[model_name][name]['base|fv'] = fe_base.predict(x).astype('float32')
        output[model_name][name]['react|fv'] = fe_react.predict(x).astype('float32')
        gc.collect()
        output[model_name][name]['base|softmax_t=1'] = get_softmax_score(model_base, x, t=1).astype('float32')
        output[model_name][name]['react|softmax_t=1'] = get_softmax_score(model_react, x, t=1).astype('float32')
        gc.collect()
        output[model_name][name]['base|softmax_t=1000'] = get_softmax_score(model_base, x, t=1000).astype('float32')
        output[model_name][name]['react|softmax_t=1000'] = get_softmax_score(model_react, x, t=1000).astype('float32')
        gc.collect()
        output[model_name][name]['base|odin_t=1000_epsilon=0.0014'] = get_odin_score(model_base, x, t=1000, epsilon=0.0014).astype('float32')
        output[model_name][name]['react|odin_t=1000_epsilon=0.0014'] = get_odin_score(model_react, x, t=1000, epsilon=0.0014).astype('float32')
        gc.collect()
        output[model_name][name]['base|odin_t=100_epsilon=0.0014'] = get_odin_score(model_base, x, t=100, epsilon=0.0014).astype('float32')
        output[model_name][name]['react|odin_t=100_epsilon=0.0014'] = get_odin_score(model_react, x, t=100, epsilon=0.0014).astype('float32')
        gc.collect()
        output[model_name][name]['base|energy'] = get_energy_score(model_base, x).astype('float32')
        output[model_name][name]['react|energy'] = get_energy_score(model_react, x).astype('float32')
        gc.collect()
    del model_base, model_react
    tf.keras.backend.clear_session()
    gc.collect()


In [ ]:
output

In [ ]:
pickle.dump(output, open('feature_vector.pkl', 'wb'))

In [ ]:
pickle.dump(weights, open('weights.pkl', 'wb'))